In [7]:
import torch
import transformers
import pandas as pd
import numpy as np

In [3]:
def read_csv(link):
    link1 = 'datasets/' + link
    link2 = '/datasets/' + link
    try:
        name_link = pd.read_csv(link1)
    except:
        name_link = pd.read_csv(link2)
    display(name_link.head()) #просмотрим сразу первые пять строк
    display(name_link.info()) #просмотрим информацию    
    display(name_link.nunique()) # посчитаем уникальные значения
    display(name_link.columns) #проверим имена столбцов
    display('Количество явных дубликатов:', name_link.duplicated().sum()) #проверка на явные дубликаты
    display(name_link.describe()) #проверим численные данные
    
    return name_link

In [4]:
df_tweets = read_csv('tweets.csv')

,text,positive
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      5000 non-null   object
 1   positive  5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


None

text        4945
positive       2
dtype: int64

Index(['text', 'positive'], dtype='object')

'Количество явных дубликатов:'

55

,positive
count,5000.00000
mean,0.50000
std,0.50005
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [12]:
# инициализируем токенизатор
tokenizer = transformers.BertTokenizer(vocab_file='datasets/vocab.txt')

In [13]:
#Чтобы токенизировать все твиты, примените метод apply():
tokenized = df_tweets['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [14]:
tokenized

0       [101, 168, 10934, 230, 11147, 241, 1985, 358, ...
1       [101, 886, 128, 888, 130, 1135, 988, 2302, 100...
2       [101, 88488, 168, 252, 13771, 31785, 247, 156,...
3       [101, 88488, 168, 10632, 37992, 11295, 3773, 1...
4       [101, 168, 10990, 11579, 230, 238, 13194, 247,...
                              ...                        
4995    [101, 168, 15627, 10978, 4502, 128, 301, 352, ...
4996    [101, 825, 1367, 3005, 949, 51158, 797, 18927,...
4997    [101, 1097, 1064, 444, 2831, 16052, 3448, 132,...
4998    [101, 1086, 1965, 128, 997, 358, 2605, 16279, ...
4999    [101, 168, 11061, 10881, 2850, 4810, 233, 5971...
Name: text, Length: 5000, dtype: object

In [15]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [16]:
padded

array([[  101,   168, 10934, ...,     0,     0,     0],
       [  101,   886,   128, ...,     0,     0,     0],
       [  101, 88488,   168, ...,     0,     0,     0],
       ...,
       [  101,  1097,  1064, ...,     0,     0,     0],
       [  101,  1086,  1965, ...,     0,     0,     0],
       [  101,   168, 11061, ...,     0,     0,     0]])

In [17]:
# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)

In [19]:
attention_mask.shape

(5000, 133)

In [ ]:
Вариант паддинга с вектором

# токенизируем текст
vector = tokenizer.encode('Очень удобно использовать уже готовый трансформатор текста', add_special_tokens=True)

# применим padding к векторам
n = 280
# англ. вектор с отступами
padded = vector + [0]*(n - len(vector))